本次作业以垃圾邮件分类任务为基础，要求提取文本特征并使用朴素贝叶斯算法进行垃圾邮件识别（调用已有工具包或自行实现）。

### 任务介绍
电子邮件是互联网的一项重要服务，在大家的学习、工作和生活中会广泛使用。但是大家的邮箱常常被各种各样的垃圾邮件填充了。有统计显示，每天互联网上产生的垃圾邮件有几百亿近千亿的量级。因此，对电子邮件服务提供商来说，垃圾邮件过滤是一项重要功能。而朴素贝叶斯算法在垃圾邮件识别任务上一直表现非常好，至今仍然有很多系统在使用朴素贝叶斯算法作为基本的垃圾邮件识别算法。

本次实验数据集来自[Trec06](https://plg.uwaterloo.ca/cgi-bin/cgiwrap/gvcormac/foo06)的中文垃圾邮件数据集，目录解压后包含三个文件夹，其中data目录下是所有的邮件（未分词），已分词好的邮件在data_cut目录下。邮件分为邮件头部分和正文部分，两部分之间一般有空行隔开。标签数据在label文件夹下，文件中每行是标签和对应的邮件路径。‘spam’表示垃圾邮件，‘ham’表示正常邮件。

本次实验

基本要求：
1. 提取正文部分的文本特征；
2. 划分训练集和测试集（可以借助工具包。一般笔记本就足够运行所有数据，认为实现困难或算力不够的同学可以采样一部分数据进行实验。）；
3. 使用朴素贝叶斯算法完成垃圾邮件的分类与预测，要求测试集准确率Accuracy、精准率Precision、召回率Recall均高于0.9（本次实验可以使用已有的一些工具包完成如sklearn）；
4. 对比特征数目（词表大小）对模型效果的影响；
5. 提交代码和实验报告。

扩展要求：
1. 邮件头信息有时也可以协助判断垃圾邮件，欢迎学有余力的同学们尝试；
2. 尝试自行实现朴素贝叶斯算法细节；
3. 尝试对比不同的概率计算方法。

### 导入工具包

In [5]:
'''
提示：
若调用已有工具包，sklearn中提供了一些可能会用到的类。
'''
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer # 提取文本特征向量的类
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, ComplementNB # 三种朴素贝叶斯算法，差别在于估计p(x|y)的方式
from sklearn.model_selection import train_test_split, GridSearchCV # 数据集划分和网格搜索交叉验证
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score # 评估指标
RANDOM_SEED = 2025 

In [16]:
import os
import numpy as np

data_dir = os.path.join(os.getcwd(),'trec06c-utf8') # 数据目录
print(os.getcwd())
# First, let's check what's in your current directory
print("Current working directory:", os.getcwd())
print("Contents:", os.listdir('.'))
label_file = os.path.join(data_dir,'label','index') # 标签文件路径

labels_array = [] # 存储标签
texts_array = [] #存储文本

with open(label_file, 'r') as f:
    lines = f.readlines()
    for line in lines:
        label,path = line.strip().split() # 分割标签和路径
        # parts = line.strip().split() # 分割标签和路径
        # label,path = parts[0],parts[1] # 获取标签和路径
        if label == 'spam':
            labels_array.append(1) # 将'spam'标签转换为1
        else:
            labels_array.append(0) # 将'ham'标签转换为0
        
        email_path = os.path.join(data_dir, path.replace('../data','data_cut')) # 构建邮件文件的完整路径
        with open(email_path,'r',encoding='utf-8',errors='ignore') as email_f:
            email_lines = email_f.readlines() # 读取邮件内容
            is_body = False # 标记是否在邮件正文部分
            email_text = [] # 存储邮件正文
            for email_line in email_lines:
                email_line = email_line.strip() # 去除行首尾空格
                if email_line == '': # 如果是空行，表示正文开始
                    is_body = True # 设置标记为True
                if is_body:
                    email_text += email_line.split() # 将正文行按空格分割并添加到email_text中
            texts_array.append(' '.join(email_text)) # 将正文列表转换为字符串并添加到texts_array中

labels = np.array(labels_array) # 转换为NumPy数组

d:\Computer_Science\御风\机器学习作业\贝叶斯垃圾邮件识别\hw3
Current working directory: d:\Computer_Science\御风\机器学习作业\贝叶斯垃圾邮件识别\hw3
Contents: ['hw3.html', 'hw3.ipynb', 'hw3.md', 'hw3.py', 'trec06c-utf8']


### 划分数据集

In [ ]:
# 划分数据集
train_texts, test_texts, train_labels, test_labels = train_test_split(texts_array, labels, test_size=0.2, random_state=RANDOM_SEED) # 划分数据集，80%训练集，20%测试集

# 从训练集中划分出10%作为验证集 (方便调参数)
train_indices, val_indices = train_test_split(np.arange(len(train_texts)),test_size=0.1, random_state=RANDOM_SEED) # 从训练集中划分出10%作为验证集

### 定义分类器模型

In [ ]:
from typing import Union,Dict
class NaiveBayes:
    def __init__(self,max_df:Union[float,int]=1.0,min_df:Union[float,int]=1,tfidf:bool = False, type:str='multinomial') -> None:
        '''
        max_df: 特征向量的最大值，默认为1.0, 超过该阈值的词语会被过滤掉
        min_df: 特征向量的最小文档频率，默认为1, 低于该阈值的词语会被过滤掉
        tfidf: 是否使用TF-IDF特征向量，默认为False
        type: 朴素贝叶斯分类器的类型，默认为'multinomial',可选值有'multinomial', 'bernoulli', 'complement'
        '''
        self.max_df = max_df
        self.min_df = min_df
        self.tfidf = tfidf
        self.type = type

    def fit(self,X,y) -> None:
        '''
        X: 文本数据
        y: 标签
        '''
        if self.tfidf:
            self.vectorizer = TfidfVectorizer(max_df=self.max_df, min_df=self.min_df) # 使用TF-IDF特征向量
        else:
            self.vectorizer = CountVectorizer(max_df=self.max_df, min_df=self.min_df) # 使用词频特征向量
        
        X = self.vectorizer.fit_transform(X) # 将文本数据转换为特征向量
        print(X.shape) # 打印特征向量的形状
        if self.type == 'multinomial':
            self.model = MultinomialNB() # 使用多项式朴素贝叶斯分类器
        elif self.type == 'bernoulli':
            self.model = BernoulliNB()
        elif self.type == 'complement':
            self.model = ComplementNB()
        else:
            raise ValueError("Invalid type, must be one of 'multinomial', 'bernoulli', 'complement'") # 如果类型不合法，抛出异常
        self.model.fit(X, y) # 训练模型
